In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
import scipy as sp
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import choice
from decision_tree.decision_tree import DecisionTree
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
%aimport decision_tree

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
%aimport

Modules to reload:
decision_tree

Modules to skip:



# Spam

In [191]:
spam = scipy.io.loadmat("spam-dataset/spam_data.mat")

In [192]:
spam_raw_train_X = spam['training_data']
spam_raw_train_y = spam['training_labels'].reshape((5172,))
spam_raw_test_X = spam['test_data']

In [193]:
from sklearn.cross_validation import train_test_split

In [206]:
a1, b1, at, bt = train_test_split(spam_raw_train_X, spam_raw_train_y, test_size=0.2)

In [207]:
print(len(a1), len(b1))

4137 1035


In [230]:
%autoreload
dt = DecisionTree({
        "max_depth":40,
        "min_points": 1
    })
dt.train(a1, at)
scored = dt.score(b1, bt)
print(sum(scored)/len(scored))

0.773913043478


In [216]:
spam_out = pd.DataFrame(dt.predict(spam_raw_test_X)).reset_index()
spam_out.head()

,index,0
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [217]:
spam_out.columns = ['Id', 'Category']
spam_out.Id = spam_out.Id + 1
spam_out.to_csv("spam_prediction.csv", index=False)

# Census

In [197]:
census = pd.read_csv("census-dataset/data.csv", na_values=["?"])
census_test = pd.read_csv("census-dataset/test_data.csv", na_values=["?"])

In [198]:
categoricals = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
non_categoricals = list(set(census.columns).difference(set(categoricals)).difference(set(["label"])))

In [199]:
census.shape

(32724, 15)

In [255]:
census_X = np.hstack([census[non_categoricals].values, pd.get_dummies(census[categoricals]).values])
census_y = census.label.values
print(census_X.shape, census_y.shape)
c1, c21, c1t, c2t = train_test_split(spam_raw_train_X, spam_raw_train_y, test_size=.2)

(32724, 105) (32724,)


In [269]:
%autoreload
dt2 = DecisionTree({
        "max_depth":2,
        "min_points": 5
    })
dt2.train(c1, c1t)

scored = dt2.score(c21, c2t)
print(sum(scored)/len(scored))

0.755555555556


In [270]:
census_test_X = np.hstack([census_test[non_categoricals].values, pd.get_dummies(census_test[categoricals]).values])

In [271]:
len(census_test_X)

16118

In [272]:
census_out = pd.DataFrame(dt2.predict(census_test_X)).reset_index()
print(census_out.head())
print(len(census_out))

   index  0
0      0  0
1      1  0
2      2  0
3      3  0
4      4  0
16118


In [273]:
census_out.columns = ['Id', 'Category']
census_out.Id = census_out.Id + 1
census_out.to_csv("census_prediction.csv", index=False)